# VacationPy
--------------
Code to get the nearest hotel in each given city and plot that on a map

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews
# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,16.61,63,100,4.63,US,1696792744
1,1,adamstown,-25.0660,-130.1015,19.29,67,19,4.09,PN,1696792792
2,2,albany,42.6001,-73.9662,16.83,52,59,0.89,US,1696792685
3,3,wailua homesteads,22.0669,-159.3780,27.24,81,0,4.12,US,1696792793
4,4,puerto natales,-51.7236,-72.4875,4.25,75,75,14.92,CL,1696792793


### Step 1: Generate a map showcasing individual points representing each city within the city_data_df DataFrame. The size of each point should correspond to the humidity level of the respective city.

In [42]:
%%capture --no-display

# Configure the map plot and Display the map
city_data_df.hvplot.points('Lng', 'Lat',size="Humidity", geo=True,color = "City", tiles='OSM')

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find ideal weather condition

In [43]:
# Narrow down cities that fit criteria and eliminate any outcomes containing null values.
city_filtered_temp = city_data_df[
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)]
# Drop any rows with null values
city_filtered_temp_df = city_filtered_temp.dropna()

# Display sample data
city_filtered_temp_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,paso de los toros,-32.8167,-56.5167,22.40,34,0,3.70,UY,1696792680
26,26,stony plain,53.5334,-114.0021,21.64,42,0,2.57,CA,1696792797
35,35,tolanaro,-25.0319,46.9987,21.77,77,0,1.67,MG,1696792799
98,98,karratha,-20.7377,116.8463,25.57,46,0,3.23,AU,1696792812
112,112,toliara,-23.3500,43.6667,24.16,87,0,2.93,MG,1696792815
174,174,lata,40.1629,-8.3327,23.84,49,0,0.68,PT,1696792828
188,188,metlili chaamba,32.2667,3.6333,25.52,24,0,2.06,DZ,1696792832
232,232,craig,40.5153,-107.5464,22.44,18,0,3.20,US,1696792841
262,262,meadow lake,34.8014,-106.5436,25.89,33,0,2.98,US,1696792847
277,277,ihosy,-22.4000,46.1167,22.53,44,0,3.14,MG,1696792850


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=city_filtered_temp_df[["City","Country","Lat","Lng","Humidity"]].copy()
hotel_df.head(2)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,paso de los toros,UY,-32.8167,-56.5167,34,
26,stony plain,CA,53.5334,-114.0021,42,
35,tolanaro,MG,-25.0319,46.9987,77,
98,karratha,AU,-20.7377,116.8463,46,
112,toliara,MG,-23.3500,43.6667,87,
174,lata,PT,40.1629,-8.3327,49,
188,metlili chaamba,DZ,32.2667,3.6333,24,
232,craig,US,40.5153,-107.5464,18,
262,meadow lake,US,34.8014,-106.5436,33,
277,ihosy,MG,-22.4000,46.1167,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 1
params = {
    "categories":categories,
    "limit":limit,
    "lang":"en",
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=hotel_df["Lat"][index]
    longitude=hotel_df["Lng"][index]
    
    #use dict param in dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
  
    # convert response to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
paso de los toros - nearest hotel: Hotel Italiana
stony plain - nearest hotel: Stony Plain Hotel
tolanaro - nearest hotel: Hôtel Mahavokey
karratha - nearest hotel: Karratha International Hotel
toliara - nearest hotel: Ambary
lata - nearest hotel: Residencial Botânico
metlili chaamba - nearest hotel: No hotel found
craig - nearest hotel: Quality Inn & Suites
meadow lake - nearest hotel: No hotel found
ihosy - nearest hotel: No hotel found
jetpur - nearest hotel: Hotel khodal
la passe - nearest hotel: L'Estuaire
calingasta - nearest hotel: Nora Hôtel
zaragoza - nearest hotel: Hotel Pilar Plaza
college station - nearest hotel: La Quinta Inn
el hadjira - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
ghadamis - nearest hotel: فندق عين الفارس
gillette - nearest hotel: Montgomery Hotel
glendive - nearest hotel: La Quinta Inn & Suites by Wyndham Glendive
saint-pierre - nearest hotel: Tropic Hotel
canmore - nearest hotel: Mountain View Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
8,paso de los toros,UY,-32.8167,-56.5167,34,Hotel Italiana
26,stony plain,CA,53.5334,-114.0021,42,Stony Plain Hotel
35,tolanaro,MG,-25.0319,46.9987,77,Hôtel Mahavokey
98,karratha,AU,-20.7377,116.8463,46,Karratha International Hotel
112,toliara,MG,-23.3500,43.6667,87,Ambary
174,lata,PT,40.1629,-8.3327,49,Residencial Botânico
188,metlili chaamba,DZ,32.2667,3.6333,24,No hotel found
232,craig,US,40.5153,-107.5464,18,Quality Inn & Suites
262,meadow lake,US,34.8014,-106.5436,33,No hotel found
277,ihosy,MG,-22.4000,46.1167,44,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
#finding all rows for which Hotel is not found
notfound_hotels_series=hotel_df["Hotel Name"]=="No hotel found"
notfound_hotels =hotel_df[notfound_hotel_series]
notfound_hotels

,City,Country,Lat,Lng,Humidity,Hotel Name
188,metlili chaamba,DZ,32.2667,3.6333,24,No hotel found
262,meadow lake,US,34.8014,-106.5436,33,No hotel found
277,ihosy,MG,-22.4000,46.1167,44,No hotel found
450,el hadjira,DZ,32.6134,5.5126,35,No hotel found
462,tsiombe,MG,-25.3000,45.4833,73,No hotel found


In [30]:
# Get the indices of the rows in filtered_df
indices_to_drop = notfound_hotels.index

#Use the drop method to eliminate the rows from the original df DataFrame
hotels_found_df = hotel_df.drop(indices_to_drop)

#display all data for which hotel is found
hotels_found_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,paso de los toros,UY,-32.8167,-56.5167,34,Hotel Italiana
26,stony plain,CA,53.5334,-114.0021,42,Stony Plain Hotel
35,tolanaro,MG,-25.0319,46.9987,77,Hôtel Mahavokey
98,karratha,AU,-20.7377,116.8463,46,Karratha International Hotel
112,toliara,MG,-23.3500,43.6667,87,Ambary
174,lata,PT,40.1629,-8.3327,49,Residencial Botânico
232,craig,US,40.5153,-107.5464,18,Quality Inn & Suites
291,jetpur,IN,21.7333,70.6167,73,Hotel khodal
295,la passe,FR,45.5549,-0.8967,44,L'Estuaire
379,calingasta,AR,-31.3308,-69.4078,4,Nora Hôtel


In [45]:
%%capture --no-display

# Configure the map plot and Display the map
hotels_found_df.hvplot.points('Lng', 'Lat',
                              size="Humidity",
                              geo=True,color = "City",
                              tiles='OSM',
                              frame_height=350,
                             hover_cols=['Hotel Name','Country'])


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)